<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/dave-updates/code/dave_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import sys
!{sys.executable} -m pip install pyspark==3.1.2 -q
!{sys.executable} -m pip install spark-nlp==4.2.0 -q

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
df = pd.read_csv('drive/MyDrive/696/text_difficulty_features.csv',sep='\t')
df.sample(25)

,original_text,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,label
47861,Counting is the action of finding the number o...,Counting be the action of find the number of e...,5.727975,5.855333,2.89,12.84,2.353125,1.43,3.66,16.0,1
368641,Essars is a commune . It is found in the regio...,Essars be a commune . It be find in the region...,6.518500,5.526875,2.89,12.05,2.442500,1.43,4.14,20.0,0
170928,"The FSF is incorporated in Massachusetts , USA .","The FSF be incorporate in Massachusetts , USA .",9.370057,6.612500,3.69,13.67,2.107500,1.43,3.00,7.0,1
293142,Age of Discovery starts .,Age of Discovery start .,8.093400,5.285000,4.37,7.84,2.400000,1.67,2.86,4.0,0
268860,Denmark is the smallest of the Scandinavian co...,Denmark be the small of the Scandinavian count...,4.344300,4.648333,3.22,7.05,2.295000,1.43,4.17,8.0,0
184692,It is the second busiest airport in Malaysia a...,It be the second busy airport in Malaysia afte...,8.187600,5.531500,3.69,8.67,2.675000,1.43,4.87,25.0,1
157618,The music video was directed by Travis Kopach ...,The music video be direct by Travis Kopach and...,8.787800,5.527000,3.69,9.68,2.894546,1.43,4.71,18.0,1
58751,"After graduating , he became involved in real ...","After graduate , he become involve in real est...",7.344964,6.205625,3.69,11.70,2.904000,1.43,4.88,17.0,1
114890,"In 1990 , June Tabor recorded an album with th...","In 1990 , June Tabor record an album with the ...",6.156133,5.459231,3.60,8.00,3.194667,1.43,4.97,18.0,1
217341,"Now , laser engraving machines are being made ...","Now , laser engrave machine be be make and eve...",5.397564,6.284545,3.69,11.56,2.643182,1.43,4.55,22.0,0


In [30]:
df.dtypes

original_text        object
lemmatized_text      object
d_chall_score       float64
aoa_mean            float64
aoa_min             float64
aoa_max             float64
conc_rating_mean    float64
conc_rating_min     float64
conc_rating_max     float64
num_lemmas          float64
label                 int64
dtype: object

In [31]:
# Extract only the columns we'll use for the classifier
df = df[df.columns[1:]]

In [9]:
from sklearn.model_selection import train_test_split

In [33]:
X = df[df.columns[:-1]]
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=.1,
                                                    stratify=y,
                                                    random_state=99)

In [36]:
X_train.shape

(355652, 9)

In [8]:
# NaiveBayes Classifier Baseline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
mnb = 

In [38]:
tf_vec = TfidfVectorizer(min_df = 20,
                         stop_words='english',
                         ngram_range=(1, 3))

In [39]:
X_train_vec = tf_vec.fit_transform(X_train['lemmatized_text'])

In [41]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'alpha': [0.001, 0.01, 0.1, 1,5,10,,50, 100]}
grid_mnb = GridSearchCV(MultinomialNB(), param_grid, scoring=)
grid_mnb.fit(X_train,y_train)